In [1]:
pip install chromadb langchain sentence-transformers huggingface_hub openai

  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached PyPika-0.48.9.tar.gz (67 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached typer-0.16.0-py3-none-any.whl.metadata (15 kB)
  Using cached kubernetes-33.1.0-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached pyproject_hooks-1.2.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached durationpy-0.10-py3-none-any.whl.metadata (340 bytes)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Us

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 4.0.0rc1 requires httpx==0.13.3, but you have httpx 0.28.1 which is incompatible.

[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text = """
Chroma is an open-source embedding database. It is used to store and query vector embeddings efficiently. 
You can use Chroma with any embedding model and LLM for building semantic search, RAG, and chatbots.
"""

# Split into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    separators=["\n\n", "\n", ".", " "]
)
chunks = text_splitter.split_text(text)
print(chunks)


['Chroma is an open-source embedding database', '. It is used to store and query vector embeddings efficiently.', 'You can use Chroma with any embedding model and LLM for building semantic search, RAG, and chatbots', '.']


In [3]:
from sentence_transformers import SentenceTransformer

# Use open-source embedding model
embed_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embed_model.encode(chunks)


c:\Users\Acer\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
pip install --upgrade langchain langchain-community chromadb sentence-transformers transformers


  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------- ----------------------- 1.0/2.5 MB 5.0 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 4.4 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 4.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/10.5 MB 5.6 MB/s eta 0:00:02
   ----- ---------------------------------- 1.3/10.5 MB 4.8 MB/s eta 0:00:02
   ---------- ----------------------------- 2.6/10.5 MB 4.3 MB/s eta 0:00:02
   -------------- ------------------------- 3.7/10.5 MB 4.3 MB/s eta 0:00:02
   ----------------- ---------------------- 4.5/10.5 MB 4.2 MB/s eta 0:00:02
   ------------------ --------------------- 4.7/10.5 MB 3.8 MB/s eta 0:00:02
   ---------------------- --


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# 📦 Required installations:
# pip install langchain chromadb sentence-transformers transformers


from langchain_community.document_loaders import TextLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline

from transformers import pipeline
import os

# 1. Sample text data
text = """Microsoft Corporation is an American multinational technology company 
headquartered in Redmond, Washington. Microsoft raised $61 million in its initial public offering in 1986.
Pando is a logistics software startup that raised $30 million in its Series B funding led by Iron Pillar.
"""

# 2. Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
docs = text_splitter.create_documents([text])

# 3. Create embeddings (using sentence-transformers)
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# 4. Store in ChromaDB
persist_directory = 'db'
vectordb = Chroma.from_documents(documents=docs,
                                 embedding=embedding,
                                 persist_directory=persist_directory)
vectordb.persist()
vectordb = None

# 5. Reload ChromaDB
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)
retriever = vectordb.as_retriever()

# 6. Load HuggingFace LLM (e.g., distilGPT2 or TinyLlama)
generator = pipeline("text-generation", model="distilgpt2", max_new_tokens=100)
llm = HuggingFacePipeline(pipeline=generator)

# 7. Build RAG QA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# 8. Utility to print results and sources
def process_llm_response(llm_response):
    print("\nAnswer:\n", llm_response['result'])
    print('\nSources:')
    for source in llm_response["source_documents"]:
        print(source.page_content)

# 🔍 Query 1
query1 = "How much money did Microsoft raise?"
response1 = qa_chain(query1)
process_llm_response(response1)

# 🔍 Query 2
query2 = "What is the news about Pando?"
response2 = qa_chain(query2)
process_llm_response(response2)


c:\Users\Acer\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Acer\AppData\Local\Temp\ipykernel_16932\557925631.py:26: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


C:\Users\Acer\AppData\Local\Temp\ipykernel_16932\557925631.py:33: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()
C:\Users\Acer\AppData\Local\Temp\ipykernel_16932\557925631.py:37: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory,
c:\Users\Acer\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Acer\.cache\huggingface\hub\models--distilgpt2.


Answer:
 Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

headquartered in Redmond, Washington. Microsoft raised $61 million in its initial public offering

Microsoft Corporation is an American multinational technology company

Pando is a logistics software startup that raised $30 million in its Series B funding led by Iron

offering in 1986.

Question: How much money did Microsoft raise?
Helpful Answer: $30 million (in the end)
Here is a quote from the Microsoft Research Center
"Microsoft's founders, John Redmond, and Joseph F. Smith, and Robert Woodruff, agreed that the company was the best company in the world in the world, and that they needed to make a profit to be able to go on a successful business."
Microsoft CEO Bill Gates also
"is one of the founders of the company, who came to be known as Microsoft, who led the way

Sources:
headquartered in Redmond, Washington